# LangChain

## Demo code to use LangChain
A quick demo code to use chains in LangChain.

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

In [ ]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from pprint import pprint
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

model = AzureChatOpenAI(deployment_name="gpt-35-turbo")

In [ ]:
tools = [
    {"type": "function", 
         "function": {
            "name": "joke", 
            "description": "Generate a joke",
            "parameters": {
                "type":"object",
                "properties": {
                    "setup": {
                        "type": "string", 
                        "description": "the setup for the joke"
                    },
                    "punchline": {
                        "type": "string", 
                        "description": "the punchline for the joke"
                    }
                },
                "required": ["setup", "punchline"]
            }
        }
    }
]

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

prompt = ChatPromptTemplate.from_template('tell me a joke about {topic}')

response = model.invoke(prompt.format(topic='programmers'))
print("1. Direct model output:")
pprint(response)
print("\n")

chain_origin = prompt | model
response = chain_origin.invoke({'topic':'programmers'})
print("2. Simple chain output:")
pprint(response)
print("\n")

chain_with_stop_bind = prompt | model.bind(stop=['?'])
response = chain_with_stop_bind.invoke({'topic':'programmers'})
print("3. Chain with stop bind output:")
pprint(response)
print("\n")

chain_with_parser = prompt | model | StrOutputParser()
response = chain_with_parser.invoke({'topic':'programmers'})
print("4. Chain with String Output Parser output:")
pprint(response)
print("\n")

chain_with_tools_bind = prompt | model.bind(tools=tools, tool_choice={'type': 'function', 'function': {'name': 'joke'}})
response = chain_with_tools_bind.invoke({'topic':'programmers'}, config={})
print("5. Chain with Tools Call output:")
pprint(response)
print("\n")

chain_with_tools_bind = prompt | model.bind(tools=tools) | JsonOutputToolsParser()
response = chain_with_tools_bind.invoke({'topic':'programmers'}, config={})
print("6. Chain with Tools Parser output:")
pprint(response)
print("\n")

In [ ]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the year {person} won the gold medal in Olmypics?")
prompt2 = ChatPromptTemplate.from_template("who's the chairman of China in the year of {year}? respond in {language}")

chain1 = prompt1 | model | StrOutputParser()
chain2 = (
    {"year": itemgetter("year"), "language": itemgetter("language")}
     | prompt2 | model | StrOutputParser()
)
chain3 = (
    {"year": chain1, "language": itemgetter("language")}
     | prompt2 | model | StrOutputParser()
)

print("1. Chain1 output:")
response = chain1.invoke({"person": "刘翔"})
pprint(response)
print("\n")

print("2. Chain2 output:")
response = chain2.invoke({"year": 2004, "language": "Chinese"})
pprint(response)
print("\n")

print("3. Chain3 output:")
response = chain3.invoke({"person": "刘翔", "language": "Chinese"})
pprint(response)
print("\n")

In [ ]:
from langchain_core.runnables.base import RunnableMap
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt1 = ChatPromptTemplate.from_template("what's year when 刘翔 won the Olmypics gold medal")
prompt2 = ChatPromptTemplate.from_template("who's the president of USA 5 years before {year}")
prompt3 = ChatPromptTemplate.from_template("who's the president of USA 5 years after {year}")
prompt4 = ChatPromptTemplate.from_template("who's more popular in USA between {president1} or {president2}")

chain1 = prompt1 | model | StrOutputParser()
chain2 = {"year":itemgetter("year")} | {
    "president1": prompt2 | model | StrOutputParser(),
    "president2": prompt3 | model | StrOutputParser()
} | prompt4 | model | StrOutputParser()
chain3 = RunnableMap({"year":chain1}) | {
    "president1": prompt2 | model | StrOutputParser(),
    "president2": prompt3 | model | StrOutputParser()
} | prompt4 | model | StrOutputParser()

print("1. Chain1 output:")
response = chain1.invoke({})
pprint(response)
print("\n")

print("2. Chain2 output:")
response = chain2.invoke({"year": 2004}, config={'callbacks': [ConsoleCallbackHandler()]})
pprint(response)
print("\n")

print("3. Chain3 output:")
response = chain3.invoke({}, config={'callbacks': [ConsoleCallbackHandler()]})
pprint(response)
print("\n")